In [1]:
import os
import json
# 其中的部分数据
filename = 'data1/4th138.json'
data = open(filename, encoding='utf-8')
strJson1 = json.load(data)


In [2]:
def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
        "Referer": "https://weibo.com"
    }
    cookies = {
        'cookie':'SINAGLOBAL=4954692280506.27.1591184066784; UOR=,,www.baidu.com; SCF=Au_QDngCAtZdjo2U3MOWDe9AqNjD2bv9bCKkhre3FM48FGdJz4rKLJEWX6ecNp7q5Tg7oJ6FfSgy02zp5_XMbKg.; ULV=1646568933516:68:1:1:5347983324126.591.1646568933470:1644499064874; SUB=_2A25PLLcYDeRhGeVO6VQV8irKyzmIHXVsW6_QrDV8PUNbmtANLRHBkW9NTWqL0HM9wbgK-_eNB3WwaFemzOLZknK_; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFd.fwHbnebHoYw4M0SwxAm5JpX5KzhUgL.Foe7eoqXeoBceh-2dJLoI7__UgSQIs8DMJxRwntt; ALF=1678375623; SSOLoginState=1646839624'
        #'cookie':'SINAGLOBAL=4954692280506.27.1591184066784; UOR=,,www.baidu.com; ULV=1640670361949:63:14:1:8632450552135.697.1640670361940:1639835996800; SCF=Au_QDngCAtZdjo2U3MOWDe9AqNjD2bv9bCKkhre3FM48Gh6mDFxMCacqDmefa19z_7fYJ9W_ldjCbGt1vKO7CaI.; SUB=_2A25Mz7rVDeRhGeVO6VQV8irKyzmIHXVvvKsdrDV8PUJbmtAKLRn-kW9NTWqL0FkKERyR5ZWAAhKgqSgGF5QPtnkp; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFd.fwHbnebHoYw4M0SwxAm5JpX5K-hUgL.Foe7eoqXeoBceh-2dJLoI7__UgSQIs8DMJxRwntt; ALF=1672280760; SSOLoginState=1640745605; XSRF-TOKEN=8rdPjDQ-y5jyz0mWRw5ZLY24'
    }
    response = requests.get(url, headers=headers, cookies=cookies)
    time.sleep(3)   # 加上5s 的延时防止被反爬
    return response.text

In [3]:
import requests
import time
header = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
        }
uid = '1226258061'
page = 0
url='https://m.weibo.cn/api/container/getIndex?uid={}&t=0&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%9B%9B%E5%B7%9D%E6%97%A5%E6%8A%A5&type=uid&value={}&containerid=107603{}&page={}'.format(uid,uid,uid,page)
URL = 'https://m.weibo.cn/api/container/getIndex?uid=3026424605&luicode=10000011&lfid=231093_-_selffollowed&type=uid&value=1222425514&containerid=1076031222425514'
html = requests.get(url = URL, headers = header)
html.text

'{"ok":0,"msg":"\\u8fd9\\u91cc\\u8fd8\\u6ca1\\u6709\\u5185\\u5bb9","data":{"cards":[]}}'

In [23]:
import requests
import re
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import matplotlib.pyplot as plt  #绘制图像的模块
import  jieba                    #jieba分词
plt.rcParams['font.sans-serif']=['SimHei'] #解决中文乱码

'''
~总思路：（找接口是真的麻烦）
0.网页版微博找了半天没找到接口，在移动端可以找到，移动端微博：https://m.weibo.cn/
1.Chrome浏览器的开发者工具下找到getIndex?网址。
2.先爬取微博ID存入列表。
3.然后找到每个ID相对应的微博url，爬取微博内容和发布时间。
4.清理数据，只剩下中文。
5.利用jieba分词。
6.画出词云。
'''
'''
函数说明：
getID():爬取微博ID
getText():爬取微博的文本即发表时间
clearText():清理文本，只保留中文
makeWordCloud():分词，绘图
'''
def getID(ID):
    print("爬取微博ID")
#    ID = []
    for p in range(0,5): #自己设置页数
        #经过尝试，在网址后面加“&page=”可以翻页
        URL = "https://m.weibo.cn/api/container/getIndex?uid=3026424605&luicode=10000011&lfid=231093_-_selffollowed&type=uid&value=1222425514&containerid=1076031222425514"+"&page="+str(p)
        try:    
            r = requests.get(url = URL, headers = header)        
            r.raise_for_status()
            r.encoding = r.apparent_encoding
            IDD = re.findall('"id":"(.*?)"',r.text) #匹配id，IDD为当前页面的所有id
            ID += IDD #将当前页面的id加入到总ID列表
            print('爬取到的id为'+ ID)
        except:
            print("爬取失败")   
    print(ID)
    return ID

def getText(ID, text):
    print("每条微博的text爬取：")
    text = []
    textsingle = []
    for i in range(len(ID)):
        print("ID:"+ID[i])
        #url后加微博ID可查看此微博的信息，其中text为文本，created_at为发表时间
        url = "https://m.weibo.cn/detail/"+ID[i]
        print(url)
        try:    
            r = requests.get(url = url, headers = header)        
            r.raise_for_status()
            r.encoding = r.apparent_encoding
            createTime = re.findall('"created_at": "(.*?)"',r.text) #匹配发表时间
            print("create at:"+createTime[0])
            print("~ ~ ~ ~ ~ ~ ~")
            textsingle = re.findall('"text": "(.*?)"',r.text) #匹配此微博文本
            text += textsingle #将此微博文本加入总微博文本列表
        except:
            print("爬取失败")
    TEXT = ''
    for i in range(len(text)):
        TEXT = TEXT + text[i] #将列表转换成字符型TEXT，便于分词
    print("爬取微博个数："+str(len(ID)))
    return TEXT

def clearText(TEXT):
    rule = re.compile(u"[^\u4e00-\u9fa5]") #匹配非中文字符
    TEXT = rule.sub('',TEXT) #将非中文字符替换为空
    return TEXT

def makeWordCloud(TEXT):    
    cut_text = " ".join(jieba.cut(TEXT)) #jieba分词，空格字符分隔
    print(cut_text)
    wordcloud = WordCloud(
       #设置字体，不然会出现口字乱码，文字的路径是电脑的字体一般路径，可以换成别的
       font_path="C:/Windows/Fonts/simfang.ttf",
       #设置了背景，宽高
       background_color="white",width=1000,height=880).generate(cut_text)
    
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

def main():
    header = {
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
        }
    ID = []
    text = []
    ID = getID(ID)
    text = getText(ID,text)
    text = clearText(text)
    makeWordCloud(text)
    
main()


爬取微博ID
爬取失败
爬取失败
爬取失败
爬取失败
爬取失败
[]
每条微博的text爬取：
爬取微博个数：0



ValueError: We need at least 1 word to plot a word cloud, got 0.

In [22]:
host = 'm.weibo.cn'
base_url = 'https://%s/api/container/getIndex?' % host
user_agent = 'User-Agent: Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B143 Safari/601.1 wechatdevtools/0.7.0 MicroMessenger/6.3.9 Language/zh_CN webview/0'#这里的user_agent是网上找的

user_id = str(3026424605)#这串数字就是用户id
headers = {
    'Host': host,
    'Referer': 'https://m.weibo.cn/u/%s'%user_id,
    'User-Agent': user_agent
}
import datetime
def timestr_standard(time_str):
    now_time = datetime.datetime.now()
    if time_str.endswith('分钟前') or time_str.endswith('小时前') or time_str == '刚刚':
        #strptime是把字符串转换为时间类。strftime是把时间转换为字符串
        time_standard = datetime.datetime.strftime(now_time.date(),'%Y-%m-%d')
    elif time_str.startswith('昨天'):
        time_standard = datetime.datetime.strftime((now_time - datetime.timedelta(days = 1)).date(),'%Y-%m-%d')
    elif time_str.startswith('0') or time_str.startswith('1'):
        time_standard = str(now_time.year) + '-' + time_str
    elif time_str.startswith('20'):
        time_standard = time_str
    return time_standard


In [23]:
import requests
import urllib
import time
import os
#import time_standard as tst
from tqdm import tqdm
from urllib.parse import urlencode
from pyquery import PyQuery as pq


In [24]:
def get_single_page(page):
    params = {
        'type': 'uid',
        'value': 1665372775,
        'containerid': int('107603' + user_id),#containerid就是微博用户id前面加上107603
        'page': page
    }
    url = base_url + urlencode(params)
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
    except requests.ConnectionError as e:
        print('抓取错误', e.args)


In [25]:
# 解析页面返回的json数据
def analysis_page(json):#保存图片的文件夹路径
    items = json.get('data').get('cards')
    for item in items:
        item = item.get('mblog')
        if item:
            data = {
                #'created_at': item.get('created_at'),#微博创建日期
                'text': pq(item.get("text")).text(),  # 仅提取内容中的文本
                #'attitudes': item.get('attitudes_count'),#点赞数
                #'comments': item.get('comments_count'),#评论数
                #'reposts': item.get('reposts_count')#转发数
            }
            base_data[len(base_data)] = data#把得到的数据字典存入总字典
#             if pic_choice == 'y':#如果选择保存图片
#                 pics = item.get('pics')
#                 if pics:
#                     for pic in pics:
#                         picture_url = pic.get('large').get('url')#得到原图地址
#                         pid = pic.get('pid')#图片id
#                         pic_name = tst.timestr_standard(data['created_at']) + '_' + pid[25:]#构建保存图片文件名，timestr_standard是一个把微博的created_at字符串转换为‘XXXX-XX-XX’形式日期的一个函数
#                         download_pics(picture_url,pic_name,pic_filebagPath)#下载原图


In [26]:
if __name__ == '__main__':
    base_data = {}
    #page = input('请输入你要爬取的页数')#可输入爬取页数，或者输入‘all’爬取所有微博
    page = 5
    #pic_choice = input('是否需要存储图片?y/n')#选择是否保存图片
    pic_choice = 'n'
    time_start=time.time()
    try:
        json = get_single_page(1)
        screen_name = json.get('data').get('cards')[0].get('mblog').get('user').get('screen_name')#博主昵称
        total = json.get('data').get('cardlistInfo').get('total')#博主微博总条数
#         if pic_choice == 'y':#如果选择保存图片，则分配图片保存路径
#             pic_filebagPath = 'D:\\python_project\\crawl\\weibo\\%s_picture'%screen_name
#             os.makedirs(pic_filebagPath)#建立文件夹
#         else:
#             pic_filebagPath = None#选择不保存文件夹则不分配路径
#         if page == 'all':#寻找总条数
#             page = total//10
#             while get_single_page(page).get('ok') == 1:
#                 page = page + 1
#             print('总页数为：%s'%page)
        page = int(page) + 1
        for page in tqdm(range(1,page)):  # 抓取数据
            json = get_single_page(page)
            analysis_page(json)
    except Exception as e:
        print('error:',e)
    finally:
        #base_dataPath = 'D:\\python_project\\crawl\\weibo\\base_data_%s.txt'%screen_name#base_data保存地址和文件名
        #f = open(base_dataPath,'w+',encoding='utf-8')
        #f.write(str(base_data))
        #f.close()
        #time_end=time.time()
        print('\n totally cost',time_end-time_start)#显示程序运行时间


100%|██████████| 5/5 [00:03<00:00,  1.36it/s]


 totally cost -1932.026831150055


In [27]:
base_data

{0: {'text': '极端分子出没 查看图片'},
 1: {'text': '诡计多端的外企 三八就你不放假 礼物是小免洗洗手液～'},
 2: {'text': '毕业论文 我真的交不出来 救救偶'},
 3: {'text': 'Repost'},
 4: {'text': '如果我有钱了…'},
 5: {'text': '今天一天都会心碎复健'},
 6: {'text': 'Repost'},
 7: {'text': '支持'},
 8: {'text': '谁呀 新年第一天 狠狠的e了'},
 9: {'text': '我觉得差不多了 大概就是以前觉得托福过了gre差不多了就是稳稳的幸福 但是现在审视整个申请过程 真正拖后腿的其实是本科院校和gpa 现在颇有少壮不努力老大徒伤悲的感觉'},
 10: {'text': '原来在水逆mdmdmdmdmd'},
 11: {'text': '左思右想还是觉得今年这个年开得有问题'},
 12: {'text': '5555如果某个学校不能update托福成绩 那我一月份考他的意义在哪'},
 13: {'text': '想到还有几个学校的推荐信链接没有发给老师 我一整个就是不想申请了'},
 14: {'text': '20岁离开得太快 对2021年的记忆居然只剩下了最后几个月不停考试的崩溃21岁生活\n2021没有那么糟糕尼'},
 15: {'text': '@ets 出下我今年的年度报告'},
 16: {'text': '求求'},
 17: {'text': '转发微博'},
 18: {'text': 'ok'},
 19: {'text': '球球了//@之之無語:🙏🙏🙏'},
 20: {'text': '球球//@muchhairdxz:我不保证了//@muchhairdxz:信女愿两个月不说脏话//@muchhairdxz:🙏 信女愿一个月不骂脏话'},
 21: {'text': '转发微博'},
 22: {'text': '我重新拼100次offer 弥补我的错误 对不起offer之神'},
 23: {'text': '我是中国人 我不做分数的奴隶'},
 24: {'text': '谢谢你'},
 25: {'text': '我不保证了//@muchhairdxz:信女